In [1]:
import numpy as np

from data_access import card_index_lookup
from lead_binary import DealMeta, seats, seat_index, suit_index_lookup

In [2]:
card_index_lookup_x = dict(
    zip(
        ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2'],
        [0, 1, 2, 3, 4, 5, 6, 7, 7, 7, 7, 7, 7],
    )
)

In [3]:
def binary_hand(suits):
    x = np.zeros(32, np.float16)
    assert(len(suits) == 4)
    for suit_index in [0, 1, 2, 3]:
        for card in suits[suit_index]:
            card_index = card_index_lookup_x[card]
            x[suit_index * 8 + card_index] += 1
    assert(np.sum(x) == 13)
    return x

In [4]:
deal_str = 'W:KQJ952.98.AQ9.95 6.AQ6.T8765.AKJ3 T874.J73.K3.8742 A3.KT542.J42.QT6'

outcome = 'N - 4S.-2.W'

play_str = 'CAC2C6C5CKC4CTC9D8D3DJDASJS6S4SAH2H8HQH3HAH7H4H9D6DKD2D9S8S3S2D5C8CQS5C3DQD7C7D4S9H6S7H5SKDTSTHTSQCJHJHK'

In [5]:
deal_meta = DealMeta.from_str(outcome)

In [6]:
declarer = 'S'
declarer_i = seat_index[declarer]
declarer_i

3

In [7]:
(declarer_i + 2) % 4

1

In [8]:
hands = list(map(lambda hand_str: list(map(list, hand_str.split('.'))), deal_str[2:].split()))

In [9]:
hands

[[['K', 'Q', 'J', '9', '5', '2'], ['9', '8'], ['A', 'Q', '9'], ['9', '5']],
 [['6'], ['A', 'Q', '6'], ['T', '8', '7', '6', '5'], ['A', 'K', 'J', '3']],
 [['T', '8', '7', '4'], ['J', '7', '3'], ['K', '3'], ['8', '7', '4', '2']],
 [['A', '3'], ['K', 'T', '5', '4', '2'], ['J', '4', '2'], ['Q', 'T', '6']]]

In [10]:
binary_hand(hands[1]).reshape((4, 8))

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  3.],
       [ 1.,  1.,  0.,  1.,  0.,  0.,  0.,  1.]], dtype=float16)

In [10]:
def get_cards(play_str):
    cards = []
    i = 0
    while i < len(play_str):
        cards.append(play_str[i:i+2])
        i += 2
    return cards

In [11]:
def get_tricks(cards):
    return list(map(list, np.array(cards).reshape((13, 4))))

In [12]:
def wins_trick_index(trick, trump, lead_index):
    led_suit = trick[0][0]
    card_values = []
    for card in trick:
        suit, value = card[0], 14 - card_index_lookup[card[1]]
        if suit == trump:
            card_values.append(value + 13)
        elif suit == led_suit:
            card_values.append(value)
        else:
            card_values.append(0)
    return (np.argmax(card_values) + lead_index) % 4

In [13]:
np.array(get_cards(play_str)).reshape((13, 4))

array([['CA', 'C2', 'C6', 'C5'],
       ['CK', 'C4', 'CT', 'C9'],
       ['D8', 'D3', 'DJ', 'DA'],
       ['SJ', 'S6', 'S4', 'SA'],
       ['H2', 'H8', 'HQ', 'H3'],
       ['HA', 'H7', 'H4', 'H9'],
       ['D6', 'DK', 'D2', 'D9'],
       ['S8', 'S3', 'S2', 'D5'],
       ['C8', 'CQ', 'S5', 'C3'],
       ['DQ', 'D7', 'C7', 'D4'],
       ['S9', 'H6', 'S7', 'H5'],
       ['SK', 'DT', 'ST', 'HT'],
       ['SQ', 'CJ', 'HJ', 'HK']],
      dtype='<U2')

In [14]:
lead_index = 0
for trick in np.array(get_cards(play_str)).reshape((13, 4)):
    win_index = wins_trick_index(trick, 'S', lead_index)
    print(trick, win_index)
    lead_index = win_index

['CA' 'C2' 'C6' 'C5'] 0
['CK' 'C4' 'CT' 'C9'] 0
['D8' 'D3' 'DJ' 'DA'] 3
['SJ' 'S6' 'S4' 'SA'] 2
['H2' 'H8' 'HQ' 'H3'] 0
['HA' 'H7' 'H4' 'H9'] 0
['D6' 'DK' 'D2' 'D9'] 1
['S8' 'S3' 'S2' 'D5'] 1
['C8' 'CQ' 'S5' 'C3'] 3
['DQ' 'D7' 'C7' 'D4'] 3
['S9' 'H6' 'S7' 'H5'] 3
['SK' 'DT' 'ST' 'HT'] 3
['SQ' 'CJ' 'HJ' 'HK'] 3


In [15]:
def get_play_labels(play_str, trump, player_turn_i):
    tricks = get_tricks(get_cards(play_str))
    
    trick_ix, leads, last_tricks, cards_in, labels = [], [], [], [], []
    
    lead_index = 0
    prev_lead_index = 0
    last_trick = ['>>', '>>', '>>', '>>']
    for trick_i, trick in enumerate(tricks):
        last_tricks.append(last_trick)
        leads.append(prev_lead_index)
        
        current_trick = ['>>', '>>', '>>']
        
        for i, card in enumerate(trick):
            player_i = (lead_index + i) % 4
            
            if player_i == player_turn_i: # the player for whom we generate data is on play
                labels.append(card)
                trick_ix.append(trick_i)
                cards_in.append(current_trick)
                break
            else:
                current_trick.append(card)
                del current_trick[0]

        if lead_index == 0:
            last_trick = trick
        elif lead_index == 1:
            last_trick = trick[3:] + trick[:3]
        elif lead_index == 2:
            last_trick = trick[2:] + trick[:2]
        else:
            last_trick = trick[1:] + trick[:1]
        prev_lead_index, lead_index = lead_index, wins_trick_index(trick, trump, lead_index)
        
    return trick_ix, leads, last_tricks, cards_in, labels

In [16]:
get_tricks(get_cards(play_str))

[['CA', 'C2', 'C6', 'C5'],
 ['CK', 'C4', 'CT', 'C9'],
 ['D8', 'D3', 'DJ', 'DA'],
 ['SJ', 'S6', 'S4', 'SA'],
 ['H2', 'H8', 'HQ', 'H3'],
 ['HA', 'H7', 'H4', 'H9'],
 ['D6', 'DK', 'D2', 'D9'],
 ['S8', 'S3', 'S2', 'D5'],
 ['C8', 'CQ', 'S5', 'C3'],
 ['DQ', 'D7', 'C7', 'D4'],
 ['S9', 'H6', 'S7', 'H5'],
 ['SK', 'DT', 'ST', 'HT'],
 ['SQ', 'CJ', 'HJ', 'HK']]

In [18]:
trick_ix, leads, last_tricks, cards_in, labels = get_play_labels(play_str, 'S', 1)
list(zip(trick_ix, leads, last_tricks, cards_in, labels))

[(0, 0, ['>>', '>>', '>>', '>>'], ['>>', '>>', 'CA'], 'C2'),
 (1, 0, ['CA', 'C2', 'C6', 'C5'], ['>>', '>>', 'CK'], 'C4'),
 (2, 0, ['CK', 'C4', 'CT', 'C9'], ['>>', '>>', 'D8'], 'D3'),
 (3, 0, ['D8', 'D3', 'DJ', 'DA'], ['>>', 'SJ', 'S6'], 'S4'),
 (4, 3, ['S6', 'S4', 'SA', 'SJ'], ['H2', 'H8', 'HQ'], 'H3'),
 (5, 2, ['HQ', 'H3', 'H2', 'H8'], ['>>', '>>', 'HA'], 'H7'),
 (6, 0, ['HA', 'H7', 'H4', 'H9'], ['>>', '>>', 'D6'], 'DK'),
 (7, 0, ['D6', 'DK', 'D2', 'D9'], ['>>', '>>', '>>'], 'S8'),
 (8, 1, ['D5', 'S8', 'S3', 'S2'], ['>>', '>>', '>>'], 'C8'),
 (9, 1, ['C3', 'C8', 'CQ', 'S5'], ['>>', 'DQ', 'D7'], 'C7'),
 (10, 3, ['D7', 'C7', 'D4', 'DQ'], ['>>', 'S9', 'H6'], 'S7'),
 (11, 3, ['H6', 'S7', 'H5', 'S9'], ['>>', 'SK', 'DT'], 'ST'),
 (12, 3, ['DT', 'ST', 'HT', 'SK'], ['>>', 'SQ', 'CJ'], 'HJ')]

In [19]:
def get_card_index(card):
    suit, value = card[0], card[1]
    return suit_index_lookup[suit] * 8 + card_index_lookup_x[value]

In [20]:
def encode_card(card):
    x = np.zeros(32, np.float16)
    if card == '>>':
        return x
    x[get_card_index(card)] = 1
    return x

In [23]:
def binary_data(deal_str, outcome_str, play_str):
    x = np.zeros((1, 11, 298), np.float16)
    y = np.zeros((1, 11, 32), np.float16)
    
    hands = list(map(lambda hand_str: list(map(list, hand_str.split('.'))), deal_str[2:].split()))
    d_meta = DealMeta.from_str(outcome_str)
    declarer_i = seat_index[d_meta.declarer]
    me_i = (declarer_i + 2) % 4

    declarer_bin = binary_hand(hands[declarer_i])
    me_bin = binary_hand(hands[me_i])
    
    _, on_leads, last_tricks, cards_ins, card_outs = get_play_labels(play_str, d_meta.strain, 1)
    
    decl_played_cards = set(['>>'])
    
    for i, (on_lead, last_trick, cards_in, card_out) in enumerate(zip(on_leads, last_tricks, cards_ins, card_outs)):
        if i > 10:
            break
        label_card_ix = get_card_index(card_out)
        y[0, i, label_card_ix] = 1
        x[0, i, 292] = d_meta.level
        if d_meta.strain == 'N':
            x[0, i, 293] = 1
        else:
            x[0, i, 294 + suit_index_lookup[d_meta.strain]] = 1
        
        x[0, i, 288 + on_lead] = 1
        
        last_trick_decl_card = last_trick[3]
        if last_trick_decl_card not in decl_played_cards:
            declarer_bin[get_card_index(last_trick_decl_card)] -= 1
            decl_played_cards.add(last_trick_decl_card)
        
        if cards_in[1] not in decl_played_cards:
            declarer_bin[get_card_index(cards_in[1])] -= 1
            decl_played_cards.add(cards_in[1])
        
        x[0, i, 32:64] = declarer_bin
        x[0, i, 0:32] = me_bin
        
        x[0, i, 64:96] = encode_card(last_trick[0])
        x[0, i, 96:128] = encode_card(last_trick[1])
        x[0, i, 128:160] = encode_card(last_trick[2])
        x[0, i, 160:192] = encode_card(last_trick[3])
        
        x[0, i, 192:224] = encode_card(cards_in[0])
        x[0, i, 224:256] = encode_card(cards_in[1])
        x[0, i, 256:288] = encode_card(cards_in[2])
        
        me_bin[label_card_ix] -= 1
        
    return x, y

In [24]:
x, y = binary_data(deal_str, outcome, play_str)

In [25]:
x.shape, y.shape

((1, 11, 298), (1, 11, 32))

In [51]:
i = 9
#y[0,i,0:32].reshape((4, 8))
x[0,i,192:288].reshape((3, 4, 8)), x[0, i, 0:32].reshape((4, 8)), x[0, i, 32:64].reshape((4, 8))

(array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]], dtype=float16),
 array([[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]], dtype=float16),
 array([[ 0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [26]:
trick_ix, leads, on_play, cards_in, labels = get_play_labels(play_str, 'S', 1)
list(zip(trick_ix, leads, on_play, cards_in, labels))

[(0, 0, ['>>', '>>', '>>', '>>'], ['>>', '>>', 'CA'], 'C2'),
 (1, 0, ['CA', 'C2', 'C6', 'C5'], ['>>', '>>', 'CK'], 'C4'),
 (2, 0, ['CK', 'C4', 'CT', 'C9'], ['>>', '>>', 'D8'], 'D3'),
 (3, 0, ['D8', 'D3', 'DJ', 'DA'], ['>>', 'SJ', 'S6'], 'S4'),
 (4, 3, ['S6', 'S4', 'SA', 'SJ'], ['H2', 'H8', 'HQ'], 'H3'),
 (5, 2, ['HQ', 'H3', 'H2', 'H8'], ['>>', '>>', 'HA'], 'H7'),
 (6, 0, ['HA', 'H7', 'H4', 'H9'], ['>>', '>>', 'D6'], 'DK'),
 (7, 0, ['D6', 'DK', 'D2', 'D9'], ['>>', '>>', '>>'], 'S8'),
 (8, 1, ['D5', 'S8', 'S3', 'S2'], ['>>', '>>', '>>'], 'C8'),
 (9, 1, ['C3', 'C8', 'CQ', 'S5'], ['>>', 'DQ', 'D7'], 'C7'),
 (10, 3, ['D7', 'C7', 'D4', 'DQ'], ['>>', 'S9', 'H6'], 'S7'),
 (11, 3, ['H6', 'S7', 'H5', 'S9'], ['>>', 'SK', 'DT'], 'ST'),
 (12, 3, ['DT', 'ST', 'HT', 'SK'], ['>>', 'SQ', 'CJ'], 'HJ')]

In [228]:
lst1 = [0, 0, 0, 0]

In [233]:
lst1.append(1)

In [236]:
lst1

[0, 0, 0, 0, 1]

In [237]:
del lst1[0]

In [238]:
lst1

[0, 0, 0, 1]